# T81-558: Applications of Deep Neural Networks
**Module 13: Advanced/Other Topics**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 13 Video Material

* Part 13.1: Flask and Deep Learning Web Services [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* **Part 13.2: Deploying a Model to AWS** [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.3: Using a Keras Deep Neural Network with a Web Application  [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.4: When to Retrain Your Neural Network [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.5: AI at the Edge: Using Keras on a Mobile Device  [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)



# Part 13.2: Deploying a Model to AWS

* [Serving TensorFlow Models](https://www.tensorflow.org/tfx/guide/serving) - Using Google's own deployment server.
* [Deploy trained Keras or TensorFlow models using Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) - Using AWS to deploy TensorFlow ProtoBuffer models.

### Exporting Model to Protocol Buffer